### layernorm

In [ ]:
# layernorm
# sqrt([1,197,768]) = [1,197,768]
# int8_matrix_A = floor(255 / threshold1 * matrix_A)
# int32_matrix_C = sqrt(int8_matrix_A)
# float32_matrix_C = sqrt(threshold1/127) * int32_matrix_C
import math
import copy
import numpy as np

def symmetrical_quantize_for_layernorm(inputs, threshold, mode="int8"):
    shape = inputs.shape
    data = inputs
    upper_coeff_dict = {"int16": 65535, "int8": 127, "int4": 15}
    upper = upper_coeff_dict[mode]
    lower = -128
    
    inv_scale = upper / threshold
    data = np.floor(data * inv_scale + 0.5)
    data = np.clip(data, lower, upper)
    # for i, d in enumerate(data):
    #     data[i] = math.floor((data[i]) * inv_scale  + 0.5)
    #     data[i] = data[i] if data[i] < upper else upper
    #     data[i] = data[i] if data[i] > lower else lower
    return data.reshape(shape)

def layernorm(x, alpha=1, beta=0):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.power(x - mean, 2) / x.shape[-1] + 1e-7
    # var = np.var(x, axis=-1, keepdims=True)
    import pdb;pdb.set_trace()
    print(var)
    return (x - mean)/var * alpha + beta

def symmetrical_dequantize_for_layernorm(inputs, threshold, mode="int8"):
    shape = inputs.shape
    data = inputs.flatten()
    coeff_dict = {"int16": 65535, "int8": 127, "int4": 15}
    
    scale = threshold1 / coeff_dict[mode]
    inv_scale = 1 / scale
    output = layernorm(inputs) * inv_scale
    return output.reshape(shape)

tensor = np.array([[[-9.0,-8.0,-7.0],[-1.9,-1.8,-1.7],[0.4,0.5,0.6],[-0.7,-0.8,-0.9]]])
# tensor = np.random.randn(10,20,30)
result = layernorm(tensor) # layernorm


print(np.max(tensor))
print(np.min(tensor))

threshold1 = np.array([9, 2, 0.6, 1]).reshape(1,-1,1)

int8_tensor = symmetrical_quantize_for_layernorm(tensor, threshold=threshold1)
int8_result = symmetrical_dequantize_for_layernorm(int8_tensor, threshold=threshold1)

result[:], int8_result[:]

# TODO: use select from table to sqrt ( replace SqrtOp and DequantizeOp with TableOP)

> /tmp/ipykernel_969950/360994248.py(31)layernorm()
     29     # var = np.var(x, axis=-1, keepdims=True)
     30     import pdb;pdb.set_trace()
---> 31     print(var)
     32     return (x - mean)/var * alpha + beta
     33 



ipdb>  np.mean(np.power(x - mean, 2), axis=-1)


array([[0.66666667, 0.00666667, 0.00666667, 0.00666667]])


ipdb>  np.var(x, axis=-1, keepdims=True)


array([[[0.66666667],
        [0.00666667],
        [0.00666667],
        [0.00666667]]])


In [49]:
x = np.array([[[-9.0,-8.0,-7.0],[-1.9,-1.8,-1.7],[0.4,0.5,0.6],[-0.7,-0.8,-0.9]]])
tensor = np.array([[[-9.0,-8.0,-7.0],[-1.9,-1.8,-1.7],[0.4,0.5,0.6],[-0.7,-0.8,-0.9]]])
# var = np.var(x, axis=-1, keepdims=True)
var = np.power(x - mean, 2) / x.shape[-1] + 1e-7
(x - mean)/var

array([[[-2.99999910e+00,  0.00000000e+00,  2.99999910e+00],
        [-2.99991000e+01,  0.00000000e+00,  2.99991000e+01],
        [-2.99991000e+01,  0.00000000e+00,  2.99991000e+01],
        [ 2.99991000e+01, -1.11022302e-09, -2.99991000e+01]]])

In [27]:
tensor.shape

(1, 4, 3)

In [24]:
var

array([[[0.66666667],
        [0.00666667],
        [0.00666667],
        [0.00666667]]])

In [17]:
import math
import numpy as np

def lowering_asymetric(data, in_scale, in_zp, upper, lower):
    for i, d in enumerate(data):
        data[i] = math.floor(data[i] / in_scale + in_zp)
        data[i] = data[i] if data[i] < upper else upper
        data[i] = data[i] if data[i] > lower else lower
    return data

def layernorm(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * np.power(x, 3))))



def getScaleAndZeropoint(fmax, fmin, qmax, qmin):
    scale = (fmax - fmin) / (qmax - qmin)
    zp = -fmin/scale + qmin
    return scale, zp

def cos_sim(a, b):
    a = np.array(a)
    b = np.array(b)
    inner_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)  
    norm_b = np.linalg.norm(b)
    cos_sim = inner_product / (norm_a * norm_b)
    return cos_sim


def table_op(tensor, in_scale, in_zp, out_scale, out_zp, qmax, qmin):
    int8_tensor = lowering_asymetric(tensor.flatten(), in_scale, in_zp, qmax, qmin)
    print(int8_tensor.astype(np.int8))
    table = []
    for i in range(qmin,qmax+1):
        step = (i - in_zp) * in_scale
        table.append(np.clip(np.floor(layernorm(step) / out_scale + out_zp), -128, 127))
    # lut
    print(len(table))
    lookup_value  = np.array([table[i+128] for i in int8_tensor.astype(np.int8)])
    return (lookup_value - out_zp) * out_scale


tensor = np.array([[-9.0,-8.0,-7.0],[-1.9,-1.8,-1.7],[49.4,49.5,49.6],[-0.7,-0.8,-0.9],[100.7,100.8,100.9]])

large_posi = np.clip(tensor, 100, 10000)

in_fmin = 0
in_fmax = 50
out_fmin = -1
out_fmax = 50
qmax = 127
qmin = -128
in_scale, in_zp = getScaleAndZeropoint(in_fmax, in_fmin, qmax, qmin)
out_scale, out_zp = getScaleAndZeropoint(out_fmax, out_fmin, qmax, qmin)
posi = table_op(np.clip(tensor, 0, 100), in_scale, in_zp, out_scale, out_zp, qmax, qmin)

in_fmin = -9
in_fmax = 0
out_fmin = -1
out_fmax = 50
qmax = 127
qmin = -128
in_scale, in_zp = getScaleAndZeropoint(in_fmax, in_fmin, qmax, qmin)
out_scale, out_zp = getScaleAndZeropoint(out_fmax, out_fmin, qmax, qmin)
neg = table_op(np.clip(tensor, -100, 0), in_scale, in_zp, out_scale, out_zp, qmax, qmin)

[-128 -128 -128 -128 -128 -128  123  124  124 -128 -128 -128  127  127
  127]
256
[-128 -100  -72   73   76   78  127  127  127  107  104  101  127  127
  127]
256


In [18]:
cos_sim(posi + neg + large_posi, np.array([layernorm(d) for d in tensor.flatten()]))

ValueError: operands could not be broadcast together with shapes (15,) (5,3) 

In [75]:
cos_sim(posi + neg, np.array([layernorm(d) for d in tensor.flatten()]))

0.9999973385304844

In [11]:
layernorm(3.4)

3.399094149016546

In [84]:
layernorm(0.5)

0.34571400982514394

In [85]:
layernorm(1)

0.8411919906082768

In [58]:
in_fmin = -9
in_fmax = 1
out_fmin = -1
out_fmax = 1
qmax = 127
qmin = -128
in_scale, in_zp = getScaleAndZeropoint(in_fmax, in_fmin, qmax, qmin)
out_scale, out_zp = getScaleAndZeropoint(out_fmax, out_fmin, qmax, qmin)
posi = table_op(np.clip(tensor, 0, 100), in_scale, in_zp, out_scale, out_zp, qmax, qmin)

[101 101 101 101 101 101 111 114 116 101 101 101]
256


In [59]:
cos_sim(posi, np.array([layernorm(d) for d in tensor.flatten()]))

0.9100494021727936

In [44]:
posi

[-0.0,
 -0.0,
 -2.3314683517128287e-15,
 -0.05454876554466734,
 -0.06474047315407029,
 -0.07589410444884379,
 0.2621611694273562,
 0.34571400982514394,
 0.43541519923081473,
 -0.16942986529488321,
 -0.1695683085635519,
 -0.16577151129027945]

array([-0.00784314, -0.00784314, -0.00784314, -0.09411765, -0.09411765,
       -0.09411765,  0.20784314,  0.29411765,  0.38039216, -0.18039216,
       -0.18039216, -0.18039216])

In [10]:
layernorm(0)

0.0

In [14]:
layernorm(10)

10.0

In [13]:
layernorm(2550)

2550.0

In [47]:
layernorm(1*8)

8.0

In [46]:
layernorm(1) * layernorm(8)

6.729535924866214

In [36]:
int8_tensor

array([[0.0140028 , 0.0280056 , 0.0420084 ],
       [0.0560112 , 0.070014  , 0.08401681],
       [0.09801961, 0.11202241, 0.12602521]])

In [24]:
tensor = np.array([[0.1,0.2,0.3],[0.4,0.5,0.6],[0.7,0.8,0.9]])
result = 0.5 * tensor * (1 + np.tanh(np.sqrt(2/np.pi) * (tensor + 0.044715 * np.power(tensor, 3))))

In [25]:
result

array([[0.05398275, 0.11585143, 0.18537092],
       [0.26216117, 0.34571401, 0.4354152 ],
       [0.53057013, 0.63043169, 0.73422849]])

In [34]:
from docx import Document
import re
from collections import Counter

def preprocess_text(text):
    # 去除标点符号和特殊字符
    text = re.sub(r"[^\w\s]", "", text)
    # 将文本转换为小写
    text = text.lower()
    return text

def merge_synonyms(word_list, synonym_mapping):
    merged_word_list = []
    for word in word_list:
        if word in synonym_mapping:
            word = synonym_mapping[word]
        merged_word_list.append(word)
    return merged_word_list

def get_top_words(word_list, n):
    word_counts = Counter(word_list)
    top_words = word_counts.most_common(n)
    return top_words

# 读取Word文档
doc = Document('intimate_relationships_text.docx')

# 提取文档内容
content = ''
for paragraph in doc.paragraphs:
    content += paragraph.text + ' '

# 预处理文本
preprocessed_content = preprocess_text(content)

# 分割文本为单词列表
word_list = preprocessed_content.split()

# 去除无用词
stop_words = ["i", "to", 'and', 'the', 'a', 'of', 'be', 'will', 'have', 'in',
                 'that', 'is', 'for', 'would', 'but', 'if', 'feel', 'it', 'with',
                 'this', 'because', 'about', 'do', 'very',  'on', 'want', 'so',
                 'them', 'think', 'they', 'as', 'their', 'or', "i'm", 'one', 'been',
                 'am', 'are', 'still', 'you', 'was', 'then', 'has', 'may','my','me','im','or','at',
              'its',]
word_list = [word for word in word_list if word not in stop_words]

# 同义词映射关系
synonym_mapping = {
    "dont want further treatment": 'give up treatment',
    'refuse the treatment': 'give up treatment',
    'opt out of the treatment': 'give up treatment',
    "not support my treatment i will quickly give up": 'give up treatment',
    'forgo treatment': 'give up treatment',
    'stop treatment': 'give up treatment',
    'not continue with the treatment': 'give up treatment',
    'stop them from curing me': 'give up treatment',
    'instead of treating my illness': 'give up treatment',
    "cant bear the thought of my family borrowing money to pay for my treatment": 'give up treatment',
    "drop the treatment": 'give up treatment',
    'give up treating me': 'give up treatment',
    'no longer treating my disease': 'give up treatment',
    'not to treat it': 'give up treatment',
    'insist on no treatment': 'give up treatment'
}

# 合并同义词
merged_word_list = merge_synonyms(word_list, synonym_mapping)

# 获取词频前50
top_words = get_top_words(merged_word_list, 50)

# 输出结果
for word, count in top_words:
    print(word, count)

family 133
not 129
life 116
dont 91
no 79
time 61
all 53
sad 53
die 48
live 46
world 44
there 44
like 43
ill 41
can 40
go 39
what 39
after 38
parents 37
treatment 35
just 33
up 33
people 33
death 33
when 33
myself 32
leave 32
friends 31
also 31
good 27
going 27
probably 26
out 25
money 25
get 24
give 23
care 23
children 23
last 22
really 22
only 22
more 22
choose 21
end 21
things 21
own 21
lonely 21
happy 21
long 21
love 21


In [33]:
!pip install python-docx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 5.6 MB 1.1 MB/s eta 0:00:01
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184600 sha256=2b718471ddbade44346ae94373fb5e3572b79bd51cac605f5ca17e050998b896
  Stored in directory: /home/user/.cache/pip/wheels/27/43/73/cb0f1fc89cb5f15ead03ed595453591f4db54fdd7dc8480f96
Successfully built python-docx
